### File structure under PeopleSnapShot


* `consensus.pkl` → the main file containing detected **SMPL pose, shape, and camera parameters** for the subject.
  * betas – the shape coefficients (10-D) describing the subject’s static body shape. Identical to standard SMPL shape parameters.
  * v_personal – a per-vertex offset array of shape (6890, 3).This stores small corrective displacements from the neutral SMPL template to the person’s fitted mesh (e.g., clothing bulges, hair, etc.).
  * When you reconstruct the canonical mesh:
  
    ```
    vertices = smpl_model(betas=betas).vertices[0] + v_personal
    ```

    you get the personalized “consensus” geometry that best fits all frames for that subject.
* `reconstructed_poses.hdf5` — stores per-frame SMPL parameters for the full sequence (same structure but over time).
  | Key         | Shape     | Meaning                                                                 |
  | ----------- | --------- | ----------------------------------------------------------------------- |
  | **`betas`** | (10,)     | Static body-shape coefficients, same for all frames.                    |
  | **`pose`**  | (649, 72) | SMPL pose parameters for each of 649 frames (24 joints × 3 axis-angle). |
  | **`trans`** | (649, 3)  | Global translation of the body root per frame.                          |

* `camera.pkl` — intrinsic/extrinsic camera parameters for projection.
* `consensus.obj` — the reconstructed mesh generated from the SMPL params.
* `tex-female-3-casual.jpg` — UV texture corresponding to the mesh.

In [5]:
pwd

'/Users/lemon/Documents/TUD/Thesis/Code/avatar-benchmark'

### SMPL params

In [ ]:
# Investigate the SMPL params from the PeopleSnapshot dataset
import os
import numpy as np
import pickle

smpl_params_dir = "data/people_snapshot_public/female-3-casual/consensus.pkl"
with open(smpl_params_dir, "rb") as f:
    smpl_params = pickle.load(f, encoding='latin1')
    
print(type(smpl_params))
print(smpl_params.keys())

for k, v in smpl_params.items():
    print(f"{k}: {type(v)}, shape: {v.shape}")

<class 'dict'>
dict_keys(['betas', 'v_personal'])
betas: <class 'numpy.ndarray'>, shape: (10,)
v_personal: <class 'numpy.ndarray'>, shape: (6890, 3)


In [ ]:
import h5py
with h5py.File('data/people_snapshot_public/female-3-casual/reconstructed_poses.hdf5', 'r') as f:
    print(list(f.keys()))  # typically ['ppose', 'trans']
    for k, v in f.items():
        print(f"{k}: {type(v)}, shape: {v.shape}")

['betas', 'pose', 'trans']
betas: <class 'h5py._hl.dataset.Dataset'>, shape: (10,)
pose: <class 'h5py._hl.dataset.Dataset'>, shape: (649, 72)
trans: <class 'h5py._hl.dataset.Dataset'>, shape: (649, 3)


In [1]:
from src.utils.data_loader import mask_loader
mask_dir = "data/people_snapshot_public/female-3-casual/masks.hdf5"
masks = mask_loader(mask_dir)
print(f"masks: {type(masks)}, shape: {masks.shape}")

masks: <class 'numpy.ndarray'>, shape: (648, 1080, 1080)


In [ ]:
import torch
pt_data = torch.load("data/people_snapshot_public/female-3-casual/poses.pt")
for k, v in pt_data.items():
    print(f"{k}: {type(v)}, shape: {v.shape}")

betas: <class 'torch.Tensor'>, shape: torch.Size([10])
thetas: <class 'torch.Tensor'>, shape: torch.Size([649, 72])
transl: <class 'torch.Tensor'>, shape: torch.Size([649, 3])


In [9]:
import numpy as np
# npz_data = np.load("../data/female-3-casual/poses.npz", allow_pickle=True)
npz_data = np.load("data/anim/aist_demo.npz", allow_pickle=True)
for k, v in npz_data.items():
    print(f"{k}: {type(v)}, shape: {v.shape}")

poses: <class 'numpy.ndarray'>, shape: (320, 72)
trans: <class 'numpy.ndarray'>, shape: (320, 3)


### Cameras

In [ ]:
# Inverstigate the camera settings
import pickle
cam = pickle.load(open("data/people_snapshot_public/female-1-casual/camera.pkl", "rb"), encoding='latin1')
for k, v in cam.items():
    print(f"{k}: {type(v)}, shape: {v.shape if hasattr(v, 'shape') else 'N/A'}")
    print(v)

camera_k: <class 'numpy.ndarray'>, shape: (5,)
[-0.11669534  0.2515035  -0.00090632 -0.00095365 -0.10548419]
camera_rt: <class 'numpy.ndarray'>, shape: (3,)
[0. 0. 0.]
camera_c: <class 'numpy.ndarray'>, shape: (2,)
[511.78055391 567.12542926]
camera_f: <class 'numpy.ndarray'>, shape: (2,)
[2664.22974522 2664.69277422]
height: <class 'int'>, shape: N/A
1080
width: <class 'int'>, shape: N/A
1080
camera_t: <class 'numpy.ndarray'>, shape: (3,)
[0. 0. 0.]


### Obj files

In [4]:
def load_shape_from_obj(file_path):
    try:
        vertices = []
        faces = []
        with open(file_path) as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#"):
                    continue

                parts = line.split()
                if len(parts) == 0:
                    continue

                if parts[0] == "v":
                    vertex = list(map(float, parts[1:]))
                    vertices.append(vertex)
                elif parts[0] == "f":
                    # Handle face indices (OBJ format can have vertex/texture/normal)
                    face = []
                    for part in parts[1:]:
                        # Split by '/' and take the first index (vertex index)
                        vertex_idx = int(part.split("/")[0])
                        face.append(vertex_idx)
                    faces.append(face)

        shape_data = {"vertices": vertices, "faces": faces}

        return shape_data

    except FileNotFoundError:
        print(f"{file_path} not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the shape: {e}")
        return None

In [5]:
# Observe SMPL-generated Mesh
# obj_path = "../data/female-4-casual/output/embedding.obj"
obj_path = "./models/smplx/smplx_uv.obj"

print("Start investigating mesh generated from SMPL parameters...")
obj_data = load_shape_from_obj(obj_path)
if obj_data:
    print(
        f"Loaded {len(obj_data['vertices'])} vertices and {len(obj_data['faces'])} faces"
    )
    print(f"First vertex: {obj_data['vertices'][0]}")
    print(f"First face: {obj_data['faces'][0]}")
else:
    print("Failed to load OBJ file")

Start investigating mesh generated from SMPL parameters...
Loaded 10475 vertices and 20908 faces
First vertex: [0.062714, 0.2885, -0.009561]
First face: [4, 2, 1]


In [52]:
# Use library to load obj file and read its content
import trimesh
obj_path = "./models/smplx/smplx_uv.obj"
mesh = trimesh.load(obj_path, process=False)
mesh.fix_normals()
print(f"Mesh has {len(mesh.vertices)} vertices, {len(mesh.faces)} faces, {len(mesh.edges)} edges, and {len(mesh.vertex_normals)} normals")
print(f"First vertex: {mesh.vertices[0]}")
print(f"First face: {mesh.faces[0]}")

Mesh has 11313 vertices, 20908 faces, 62724 edges, and 11313 normals
First vertex: [ 0.062714  0.2885   -0.009561]
First face: [3 1 0]


### Explanation of the obj file

In the vertices section it stores the position of every vertices, then face section stores the three vertices it use to define the face. So the bary-centric center is the average position of the three vertices.

In [ ]:
# Observe Mesh ==> Gaussian Mapping
import json
json_path = "../data/female-4-casual/output/embedding.json"

print("Investigating mapping from mesh to Gaussian...")
json_data = json.load(open(json_path))
print(f"Loaded JSON data with keys: {list(json_data.keys())}")
for key in json_data:
    print(
        f"{key}: {type(json_data[key]), len(json_data[key]) if hasattr(json_data[key], '__len__') else 'N/A'}"
    )
    
# List out the cano_mesh elements
cano_mesh = json_data.get("cano_mesh", {})
if cano_mesh:
    print(cano_mesh, type(cano_mesh))

Investigating mapping from mesh to Gaussian...
Loaded JSON data with keys: ['cano_mesh', 'sample_fidxs', 'sample_bary', '_xyz', '_rotation']
cano_mesh: (<class 'str'>, 13)
sample_fidxs: (<class 'list'>, 106073)
sample_bary: (<class 'list'>, 106073)
_xyz: (<class 'list'>, 106073)
_rotation: (<class 'list'>, 106073)
embedding.obj <class 'str'>


In [5]:
# Observe Animation file structure(.npz)
# It covers SMPL parameters per frame
import numpy as np 
npz_path = "../data/female-3-casual/poses/anim_nerf_test.npz"

print("Investigating animation .npz file structure...")
npz_data = np.load(npz_path)
print(f"Loaded .npz file with keys: {list(npz_data.keys())}")
for key in npz_data:
    print(
        f"{key}: {type(npz_data[key]), npz_data[key].shape if hasattr(npz_data[key], 'shape') else 'N/A'}"
    )   

Investigating animation .npz file structure...
Loaded .npz file with keys: ['betas', 'global_orient', 'body_pose', 'transl']
betas: (<class 'numpy.ndarray'>, (1, 10))
global_orient: (<class 'numpy.ndarray'>, (51, 3))
body_pose: (<class 'numpy.ndarray'>, (51, 69))
transl: (<class 'numpy.ndarray'>, (51, 3))


### Invertigate ply file

In [82]:
from plyfile import PlyData, PlyElement

def load_ply_data(ply_file_path):
    return PlyData.read(ply_file_path)

def inspect_ply_file(ply_file_path, redundant_info=False, k=5):
    plydata = load_ply_data(ply_file_path)
    print(f"Number of elements: {len(plydata.elements)}")
    for element in plydata.elements:
        print(f"Element: {element.name}, count: {element.count}")
        if redundant_info:
            for property in element.properties:
                print(f"  Property: {property.name}, type: {property.dtype}, values: {plydata[element.name][property.name][:k]}")  # Print first 5 values
    return plydata

def check_value(ply_file_path, key):
    plydata = load_ply_data(ply_file_path)
    data = plydata["vertex"][key]
    print(f"Values for key '{key}':")
    for val in data:
        print(val)
    return

def check_value_zero(ply_file_path, key):
    plydata = load_ply_data(ply_file_path)
    data = plydata["vertex"][key]
    flag = False
    for val in data:
        if val != 0:
            flag = True
            print(val)
    if not flag:
        print("All zero for key:", key)

In [85]:
std_data = inspect_ply_file("./data/SplattingAvatar/female-4-casual/output/point_cloud.ply", True)

Number of elements: 1
Element: vertex, count: 106073
  Property: x, type: <bound method PlyProperty.dtype of PlyProperty('x', 'float')>, values: [0.1229206  0.11025127 0.14186215 0.5114285  0.2984839 ]
  Property: y, type: <bound method PlyProperty.dtype of PlyProperty('y', 'float')>, values: [-0.60400337 -0.57892483 -0.3726355  -0.12982011  0.8164872 ]
  Property: z, type: <bound method PlyProperty.dtype of PlyProperty('z', 'float')>, values: [5.258458  5.2658134 5.3073277 5.440744  5.5729604]
  Property: nx, type: <bound method PlyProperty.dtype of PlyProperty('nx', 'float')>, values: [0. 0. 0. 0. 0.]
  Property: ny, type: <bound method PlyProperty.dtype of PlyProperty('ny', 'float')>, values: [0. 0. 0. 0. 0.]
  Property: nz, type: <bound method PlyProperty.dtype of PlyProperty('nz', 'float')>, values: [0. 0. 0. 0. 0.]
  Property: f_dc_0, type: <bound method PlyProperty.dtype of PlyProperty('f_dc_0', 'float')>, values: [ 0.9948274  -1.7787342  -0.5159113   0.43796745  0.8818083 ]
  P

In [86]:

scales = std_data['vertex']['scale_1']
scales.sort()
print("Std scales:", scales)

# check_value_zero("./data/SplattingAvatar/female-4-casual/output/point_cloud.ply", "nx")
# check_value_zero("./data/SplattingAvatar/female-4-casual/output/point_cloud.ply", "ny")
# check_value_zero("./data/SplattingAvatar/female-4-casual/output/point_cloud.ply", "nz")

Std scales: [-18.13021   -18.129055  -18.121668  ...  -2.5352063  -2.4325366
  -2.316835 ]


In [88]:
inspect_ply_file("./models/avatar_template.ply", True)

Number of elements: 1
Element: vertex, count: 83632
  Property: x, type: <bound method PlyProperty.dtype of PlyProperty('x', 'float')>, values: [0.06419333 0.064933   0.062892   0.064755   0.074861  ]
  Property: y, type: <bound method PlyProperty.dtype of PlyProperty('y', 'float')>, values: [0.288613   0.2886695  0.2891655  0.288004   0.29270732]
  Property: z, type: <bound method PlyProperty.dtype of PlyProperty('z', 'float')>, values: [-0.009131   -0.008916   -0.009434   -0.009043   -0.01774133]
  Property: opacity, type: <bound method PlyProperty.dtype of PlyProperty('opacity', 'float')>, values: [0.6 0.6 0.6 0.6 0.6]
  Property: f_dc_0, type: <bound method PlyProperty.dtype of PlyProperty('f_dc_0', 'float')>, values: [0.5 0.5 0.5 0.5 0.5]
  Property: f_dc_1, type: <bound method PlyProperty.dtype of PlyProperty('f_dc_1', 'float')>, values: [0.5 0.5 0.5 0.5 0.5]
  Property: f_dc_2, type: <bound method PlyProperty.dtype of PlyProperty('f_dc_2', 'float')>, values: [0.5 0.5 0.5 0.5 0.5

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('opacity', 'float'), PlyProperty('f_dc_0', 'float'), PlyProperty('f_dc_1', 'float'), PlyProperty('f_dc_2', 'float'), PlyProperty('scale_0', 'float'), PlyProperty('scale_1', 'float'), PlyProperty('scale_2', 'float'), PlyProperty('rot_0', 'float'), PlyProperty('rot_1', 'float'), PlyProperty('rot_2', 'float'), PlyProperty('rot_3', 'float')), count=83632, comments=[]),), text=False, byte_order='<', comments=[], obj_info=[])

In [68]:
inspect_ply_file("./data/gaussian_scene/flowers.ply", True)

Number of elements: 1
Element: vertex, count: 3636448
  Property: x, type: <bound method PlyProperty.dtype of PlyProperty('x', 'float')>, values: [ 0.24265061  0.19423085 -5.913421    0.81245714  2.04914   ]
  Property: y, type: <bound method PlyProperty.dtype of PlyProperty('y', 'float')>, values: [-0.32327402  0.2511039   1.4171636   0.15862544  0.5046454 ]
  Property: z, type: <bound method PlyProperty.dtype of PlyProperty('z', 'float')>, values: [-0.0578796  -0.19776651  1.3705724   0.2141829  -0.5727541 ]
  Property: nx, type: <bound method PlyProperty.dtype of PlyProperty('nx', 'float')>, values: [0. 0. 0. 0. 0.]
  Property: ny, type: <bound method PlyProperty.dtype of PlyProperty('ny', 'float')>, values: [0. 0. 0. 0. 0.]
  Property: nz, type: <bound method PlyProperty.dtype of PlyProperty('nz', 'float')>, values: [0. 0. 0. 0. 0.]
  Property: f_dc_0, type: <bound method PlyProperty.dtype of PlyProperty('f_dc_0', 'float')>, values: [-0.47322437 -1.1650956   0.25865018  1.1916084  

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('nx', 'float'), PlyProperty('ny', 'float'), PlyProperty('nz', 'float'), PlyProperty('f_dc_0', 'float'), PlyProperty('f_dc_1', 'float'), PlyProperty('f_dc_2', 'float'), PlyProperty('f_rest_0', 'float'), PlyProperty('f_rest_1', 'float'), PlyProperty('f_rest_2', 'float'), PlyProperty('f_rest_3', 'float'), PlyProperty('f_rest_4', 'float'), PlyProperty('f_rest_5', 'float'), PlyProperty('f_rest_6', 'float'), PlyProperty('f_rest_7', 'float'), PlyProperty('f_rest_8', 'float'), PlyProperty('f_rest_9', 'float'), PlyProperty('f_rest_10', 'float'), PlyProperty('f_rest_11', 'float'), PlyProperty('f_rest_12', 'float'), PlyProperty('f_rest_13', 'float'), PlyProperty('f_rest_14', 'float'), PlyProperty('f_rest_15', 'float'), PlyProperty('f_rest_16', 'float'), PlyProperty('f_rest_17', 'float'), PlyProperty('f_rest_18', 'float'), PlyProperty('f_rest_19', 'float'), PlyProperty('f_res